In [46]:
import os
import psycopg2
from dotenv import load_dotenv
import pandas as pd
import numpy as np

In [47]:
load_dotenv()

conn = psycopg2.connect(
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT"),
    database=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD")
)

In [48]:
cur = conn.cursor()
cur.execute("SELECT version();")
print(cur.fetchone())

('PostgreSQL 16.12 (Debian 16.12-1.pgdg13+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 14.2.0-19) 14.2.0, 64-bit',)


In [49]:
cur.execute("""
CREATE TABLE IF NOT EXISTS aircraft (
    icao24 CHAR(6) PRIMARY KEY,
    callsign VARCHAR(8)
);
""")

In [50]:
df = pd.read_csv("../data/clean/opensky_asia_features.csv")

aircraft_df = df[["icao24", "callsign"]].drop_duplicates()

for _, row in aircraft_df.iterrows():
    cur.execute(
        """
        INSERT INTO aircraft (icao24, callsign)
        VALUES (%s, %s)
        ON CONFLICT (icao24) DO NOTHING;
        """,
        (row["icao24"], row["callsign"])
    )

conn.commit()

In [51]:
cur.execute("""
CREATE TABLE IF NOT EXISTS aircraft_states (
    id BIGSERIAL PRIMARY KEY,

    icao24 CHAR(6) REFERENCES aircraft(icao24),
    timestamp TIMESTAMP NOT NULL,

    latitude DOUBLE PRECISION,
    longitude DOUBLE PRECISION,
    baro_altitude DOUBLE PRECISION,

    velocity DOUBLE PRECISION,
    vertical_rate DOUBLE PRECISION,
    true_track DOUBLE PRECISION,
    on_ground BOOLEAN,

    track_sin DOUBLE PRECISION,
    track_cos DOUBLE PRECISION,

    delta_time DOUBLE PRECISION,
    distance_delta DOUBLE PRECISION,
    acceleration DOUBLE PRECISION,
    turn_rate DOUBLE PRECISION,

    climb_phase INTEGER,
    speed_bucket TEXT
);
""")

In [52]:
cur.execute("""
CREATE INDEX IF NOT EXISTS idx_states_icao_time
ON aircraft_states (icao24, timestamp);
""")

cur.execute("""
CREATE INDEX IF NOT EXISTS idx_states_time
ON aircraft_states (timestamp);
""")

conn.commit()

In [56]:
from psycopg2.extras import execute_batch

state_cols = [
    "icao24", "timestamp",
    "latitude", "longitude", "baro_altitude",
    "velocity", "vertical_rate", "true_track", "on_ground",
    "track_sin", "track_cos",
    "delta_time", "distance_delta",
    "acceleration", "turn_rate",
    "climb_phase", "speed_bucket"
]

records = list(df[state_cols].itertuples(index=False, name=None))

execute_batch(
    cur,
    """
    INSERT INTO aircraft_states (
        icao24, timestamp,
        latitude, longitude, baro_altitude,
        velocity, vertical_rate, true_track, on_ground,
        track_sin, track_cos,
        delta_time, distance_delta,
        acceleration, turn_rate,
        climb_phase, speed_bucket
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """,
    records,
    page_size=5000
)

conn.commit()

In [62]:
conn.rollback()

In [65]:
cur.execute("SELECT COUNT(*) FROM aircraft_states LIMIT 10;")
print(cur.fetchall())

[(1058291,)]


In [66]:
cur.close()
conn.close()